In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import time
import wandb
import pytorch_lightning as pl

import FunctionCollection as fc
import importlib
fc = importlib.reload(fc)
import os

In [2]:
path = r'C:\Users\jv97\Desktop\github\Neutrino-Machine-Learning'

run_name = 'oscNext_angle_m9'

args = {'N_edge_feats': 6,
        'N_dom_feats': 5,
        'N_targets': 2,
        'N_outputs': 4,
        'N_metalayers': 1,
        'N_hcs': 64,
        'diagonal_cov': True,
        'wandb_activated': True,
        'type': 'twice_Polar_NLLH',
        'zenith': True,
        'id': '2vr5',#wandb.util.generate_id()[:4],
        'eps': 0,
        'lr': 0.0209,
        'filename': 'dev_level7_oscNext_IC86_003_unscaled.db',#dev_level7_mu_e_tau_oscweight_000.db #rasmus_classification_muon_3neutrino_3mio.db #dev_level7_oscNext_IC86_003.db
        'features': 'charge_log10, dom_time, dom_x, dom_y, dom_z',
        'targets': 'event_no, azimuth_retro, zenith_retro, azimuth_sigma, zenith_sigma',
        'TrTV': (0,1,1)
       }

In [3]:
tf = pd.read_pickle(r'C:\Users\jv97\Desktop\github\Neutrino-Machine-Learning\datasets\transformers.pkl')
time_center, time_scale = tf['features']['time'].center_[0], tf['features']['time'].scale_[0]
charge_center, charge_scale = tf['features']['charge_log10'].center_[0], tf['features']['charge_log10'].scale_[0]

In [4]:
filepath = os.path.join(path,'raw_data/dev_level7_oscNext_IC86_003/data')
# event_nos = pd.read_pickle(os.path.join(path,f'checkpoints/{run_name}_{args["id"]}/test_event_nos.pkl')).iloc[:1000000]

def x_transform(df):
    df['charge_log10'] = (df['charge_log10'] - charge_center)/charge_scale
    df['dom_time'] = (df['dom_time'] - time_center)/time_scale
    df[['dom_x','dom_y','dom_z']] /= 300
    return torch.tensor(df.values)
def y_transform(df):
    return torch.tensor(df.values)

dataset = fc.custom_db_dataset(filepath = filepath,
                               filename = args['filename'],
                               features = args['features'],
                               targets = args['targets'],
                               TrTV = args['TrTV'],
#                                event_nos = event_nos,
                               x_transform = x_transform,
                               y_transform = y_transform,
                               shuffle = False)

In [5]:
# train_loader, test_loader, val_loader = dataset.return_dataloaders(batch_size=512) #~0.6sec loading time pr. batch
data_loader = dataset.return_dataloader(batch_size=1024)

In [6]:
import Model_Loaders.Model_9 as M
M = importlib.reload(M)

Net = M.Load_model(args['type'],args)
model = Net.load_from_checkpoint(path + '/checkpoints/' + run_name + '_' + args['id'] + '/' + run_name + '_latest.ckpt')

Remember all accuracies are positive and defined to go towards 0 in the optimal case.


In [7]:
model.to(torch.device('cuda'))
pred, truth = fc.return_reco_truth(model,data_loader)

Batch: 100%|███████████████████████████████████| 63/63 [02:14<00:00,  1.73s/it]

In [8]:
df_truth = pd.DataFrame(data=truth,columns=args['targets'].split(', '))
df_pred = pd.DataFrame(data=pred,columns=['azimuth_pred','zenith_pred','azimuth_pred_sigma','zenith_pred_sigma'])
# pd.concat([df_truth,df_pred],axis=1).to_pickle(r'C:\Users\jv97\Desktop\github\Neutrino-Machine-Learning\checkpoints\oscNext_angle_m9_2vr5\test_data_predictions.pkl')

In [9]:
pd.concat([df_truth,df_pred],axis=1)

,event_no,azimuth_retro,zenith_retro,azimuth_sigma,zenith_sigma,azimuth_pred,zenith_pred,azimuth_pred_sigma,zenith_pred_sigma
0,0.0,1.947995,1.447643,0.011919,0.006036,2.097930,1.554986,-7.287058,-8.975826
1,1.0,1.429471,0.964943,0.313485,0.461077,1.212115,1.022235,-0.896617,-2.218345
2,2.0,2.910558,0.779122,0.786920,0.352431,3.311777,1.598022,-1.387376,-2.446280
3,3.0,4.711105,1.321742,0.463091,0.317207,4.021784,1.779897,-0.627772,-2.152980
4,4.0,2.461376,1.561457,0.473049,0.526679,1.738041,1.736209,-0.974840,-2.288333
...,...,...,...,...,...,...,...,...,...
63796,117702055.0,4.581205,0.322708,2.936832,0.518159,0.218523,1.432123,-0.524001,-1.874680
63797,117702056.0,5.190562,1.688600,0.346256,0.168343,0.392853,1.405627,-0.554134,-2.678306
63798,117702057.0,0.176816,1.671801,-5.833329,0.307980,0.449605,1.493591,-0.989682,-2.247345
63799,117702058.0,3.374813,0.751226,1.782722,0.387041,3.359108,1.503775,-0.705107,-1.928802


In [103]:
# wandb.finish()

_runtime,14458
_timestamp,1616003189
_step,10749
Train Loss,-0.33702
Train Acc,0.6416
epoch,27
Val Acc,0.65624
lr-Adam,0.00678


_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Train Loss,█▆▆▆▅▄▅▅▅▄▃▃▃▃▃▃▃▂▂▂▃▃▄▂▂▂▂▂▂▂▂▂▂▁▁▃▁▂▂▁
Train Acc,█▆▆▆▆▅▅▆▅▃▃▂▂▂▂▂▂▁▁▂▂▂▃▁▁▂▂▁▁▁▁▁▂▁▁▂▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
Val Acc,██▇▇▇▆▃▂▃▂▂▁▁▁▂▂▁▁▁▁▁▂▁▁▁▁▁
lr-Adam,▁█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂
